In [1]:
function snailfish_explode(num)
    exploded = false
    counter = 0
    for (i, c) in enumerate(num)
        counter += c == '[' ? 1 : 0
        counter -= c == ']' ? 1 : 0
        if counter == 5
            end_explode = findnext(collect(num) .== ']', i)
            i = findlast(collect(num[1:end_explode]) .== '[')
            to_explode = num[i:end_explode]
            el, er = parse.(Int, split(to_explode[2:end-1], ','))
            left = num[1:i-1]
            # No findlast(::RegEx, ...) for some reason
            inds_left = findall(r"\d+", left)
            if length(inds_left) >= 1
                inds_left = inds_left[end]
                value = parse(Int, left[inds_left]) + el
                left = left[1:inds_left.start-1] * "$value" * left[inds_left.stop+1:end]
            end
            right = num[end_explode+1:end]
            inds_right = findfirst(r"\d+", right)
            if inds_right != nothing
                value = parse(Int, right[inds_right]) + er
                right = right[1:inds_right.start-1] * "$value" * right[inds_right.stop+1:end]
            end
            num = left * '0' * right
            exploded = true
            break
        end
    end
    return exploded, num
end

function snailfish_split(num)
    inds = findfirst(r"\d{2}", num)
    if inds == nothing
        return false, num
    else
        value = parse(Int, num[inds])
        new = "[$(value ÷ 2),$(Int(ceil(value / 2)))]"
        num = num[1:inds.start-1] * new * num[inds.stop+1:end]
        return true, num
    end
end

function split_pair(num)
    counter, i= 0, 0
    for (i, c) in enumerate(num)
        if c==',' && counter == 1
            return num[2:i-1], num[i+1:end-1]
        end
        counter += c == '[' ? 1 : 0
        counter -= c == ']' ? 1 : 0
    end
end

function magnitude(num)
    if ',' ∉ num
        return parse(Int, num)
    else
        p = split_pair(num)
        return 3*magnitude(p[1]) + 2*magnitude(p[2])
    end
end

function snailfish_reduce(num)
    actions_done = true
    while actions_done
        expl, num = snailfish_explode(num)
        if expl
            continue
        end
        split, num = snailfish_split(num)
        actions_done = expl | split
    end
    return num
end

function snailfish_add(first, second)
    out = *('[', first, ',', second, ']')
    return snailfish_reduce(out)
end

snailfish_add (generic function with 1 method)

In [2]:
function solve_part1(filename)
    lines = readlines(filename)
    return magnitude(reduce(snailfish_add, lines))
end
println("Part 1 test: $(solve_part1("test.txt"))")
println("Part 1 solution: $(solve_part1("input.txt"))")

Part 1 test: 4140
Part 1 solution: 4017


In [3]:
function solve_part2(filename)
    mags = []
    lines = readlines(filename)
    for l1 in lines
        for l2 in lines
            if l1 != l2
                push!(mags, magnitude(snailfish_add(l1, l2)))
            end
        end
    end
    return maximum(mags)
end
println("Part 2 test: $(solve_part2("test.txt"))")
println("Part 2 solution: $(solve_part2("input.txt"))")

Part 2 test: 3993
Part 2 solution: 4583
